In [14]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline
import cairosvg
import os
import random
import json
import os
import time
from PIL import Image
import cv2
import numpy as np
from schemdraw.parsing import logicparse
def convert_svg_to_png(svg_file, png_file):
    cairosvg.svg2png(url=svg_file, write_to=png_file)


In [3]:
data_path='./circuits_output/'
for file in os.listdir(data_path):
    if file.endswith('.svg'):
        convert_svg_to_png(data_path+file, data_path+file.replace('.svg', '.png'))


In [15]:
def load_image(image_path):
    # load the image return the 2D numpy array.
    return np.array(Image.open(image_path))[:, :, 3]

In [19]:
def get_bbox(img, template_path='./'):
    """
    Generate bounding boxes for logic gates in the given image.
    :param image_path: Path to the PNG image of the circuit.
    :return: List of bounding boxes in the format [(x1, y1, x2, y2, gate_type), ...].
    """

    # Define templates for logic gates and their corresponding colors
    templates = {
        "and": np.array(Image.open(os.path.join(template_path,'and.png')))[:, :, 3],
        "or": np.array(Image.open(os.path.join(template_path,'or.png')))[:, :, 3],
        "not": np.array(Image.open(os.path.join(template_path,'not.png')))[:, :, 3],
        "xor": np.array(Image.open(os.path.join(template_path,'xor.png')))[:, :, 3],
        "notand": np.array(Image.open(os.path.join(template_path,'notand.png')))[:, :, 3],
        "notor": np.array(Image.open(os.path.join(template_path,'notor.png')))[:, :, 3],
        "notxor": np.array(Image.open(os.path.join(template_path,'notxor.png')))[:, :, 3],
    }

    thresholds = {
        "and": 0.90,
        "or": 0.90,
        "not": 0.92,
        "xor": 0.86,
        "notand": 0.84,
        "notor": 0.84,
        "notxor": 0.84,
    }
    bboxes = []

    def calculate_iou(box1, box2):
        # Extract coordinates
        x1, y1, x2, y2 = box1
        x1_p, y1_p, x2_p, y2_p = box2

        # Calculate intersection coordinates
        inter_x1 = max(x1, x1_p)
        inter_y1 = max(y1, y1_p)
        inter_x2 = min(x2, x2_p)
        inter_y2 = min(y2, y2_p)

        # Calculate intersection area
        inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

        # Calculate areas of both boxes
        box1_area = (x2 - x1) * (y2 - y1)
        box2_area = (x2_p - x1_p) * (y2_p - y1_p)

        # Calculate union area
        union_area = box1_area + box2_area - inter_area

        # Avoid division by zero
        if union_area == 0:
            return 0

        # IoU
        return inter_area / union_area

    # Perform template matching for each gate
    for gate_type, template in templates.items():
        result = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
        loc = np.where(result >= thresholds[gate_type])
        
        # Generate bounding boxes
        for pt in zip(*loc[::-1]):  # Reverse coordinates
            x1, y1 = pt
            x2, y2 = x1 + template.shape[1], y1 + template.shape[0]
            
            # Apply Non-Maximum Suppression (NMS)
            add_box = True
            for existing_box in bboxes:
                if calculate_iou((x1, y1, x2, y2), existing_box[:4]) > 0.5:  # IoU threshold
                    add_box = False
                    break
            if add_box:
                if int(x1)<50:
                    x1=0
                bboxes.append((int(x1), int(y1), int(x2), int(y2), gate_type))

    return bboxes

In [20]:
def generate_fixed_depth_expression(inputs, depth):
    """
    Generate a boolean expression with exactly the specified depth.
    Avoid consecutive 'not' operations and redundant nesting.
    """
    operators = ["and", "or", "xor"]  # Binary operators
    symbols = [chr(65 + i) for i in range(inputs)]  # A, B, C, D...

    def generate_expr(current_depth, allow_not=True):
        if current_depth == depth:
            # Base case: choose a symbol when depth is reached
            return random.choice(symbols)
        else:
            op = random.choice(operators + (["not"] if allow_not else []))
            if op == "not":
                # Avoid consecutive 'not' by disallowing 'not' in nested call
                # if current_depth < depth - 1:
                #     exp = f"not {generate_expr(current_depth, allow_not=False)}"
                # else:
                #     exp = f"not {generate_expr(current_depth + 1, allow_not=False)}"
                # return exp
                if current_depth < depth - 1:
                    exp = generate_expr(current_depth, allow_not=False)
                else:
                    exp = generate_expr(current_depth + 1, allow_not=False)
                if exp.startswith('(not') or exp.startswith('not '):
                    return exp
                else:
                    return f'not {exp}'
            else:
                # Binary operators increase depth
                left = generate_expr(current_depth + 1)
                right = generate_expr(current_depth + 1)
                # Ensure left and right are distinct
                while left == right:
                    right = generate_expr(current_depth + 1)
                # Add parentheses around left and right if they are not at the deepest layer
                if current_depth + 1 < depth:
                    left = f"({left})"
                    right = f"({right})"
                return f"{left} {op} {right}"

    return generate_expr(0)

def create_circuit_and_metadata(output_dir, num_circuits, inputs, depth, start_index=1):
    """
    Create circuits and save metadata and diagrams as individual JSON and SVG files.
    """
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_circuits):
        circuit_id = f"circuit_{i + start_index}"
        print(f"Generating {circuit_id} ...")
        expression = generate_fixed_depth_expression(inputs, depth)
        # expression = "(not (((C or A) xor (C and C))))"
        print(expression)
        

        # Create and save the circuit diagram
        try:
            print("Start parsing ...")
            tik = time.time()
            drawing = logicparse(expression, gateH=1.2)
            
            svg_file = os.path.join(output_dir, f"{circuit_id}.svg")
            drawing.save(svg_file)
            convert_svg_to_png(svg_file, svg_file.replace('.svg', '.png'))
            bbox= get_bbox(load_image(svg_file.replace('.svg', '.png')))
            print(bbox)
            print(f"Parsing finished in {time.time() - tik} s")
            metadata_entry = {
                "circuit_id": circuit_id,
                "expression": expression,
                "inputs": [chr(65 + j) for j in range(inputs)],  # A, B, C, D...
                "depth": depth,
                "bbox": bbox
            }
    
            # Save metadata to individual JSON file
            metadata_file = os.path.join(output_dir, f"{circuit_id}.json")
            with open(metadata_file, "w") as f:
                json.dump(metadata_entry, f, indent=4)
        except Exception as e:
            print(f"Error generating circuit diagram for expression {expression}: {e}")



In [37]:
# test only
t='((not (not B)) xor (not (not A)))'
name='xor'
drawing = logicparse(t, gateH=1.2)
drawing.save(f'./{name}.svg')
convert_svg_to_png(f'./{name}.svg', f'./{name}.png')

In [38]:
from PIL import Image

# Load the image
image_path = "./xor.png"
img = Image.open(image_path)

# Define the cropping box (left, upper, right, lower)
# Adjust these values to remove the "B"
crop_box = (0, 0, img.width-15, img.height)  # Example: crop 20 pixels from the left

# Crop the image
cropped_img = img.crop(crop_box)
cropped_img.save(image_path)

In [21]:
# Parameters
output_directory = "./circuits_output2"
number_of_circuits = 40  # Number of circuits to generate
number_of_inputs = [2,3]     # Input variables (e.g., A, B, C, D)
fixed_depth = [2,3,4]          # Fixed depth for all expressions
start_index = 1
random.seed(0)
for inputs in number_of_inputs:
    for depth in fixed_depth:
        create_circuit_and_metadata(output_directory, number_of_circuits, inputs, depth, start_index)
        start_index += number_of_circuits

Generating circuit_1 ...
not (B and A) or (B or A)
Start parsing ...
[(0, 0, 111, 60, 'and'), (128, 58, 201, 117, 'or'), (0, 116, 105, 175, 'or')]
Parsing finished in 0.06255602836608887 s
Generating circuit_2 ...
(A xor B) and (A and B)
Start parsing ...
[(125, 57, 206, 117, 'and'), (0, 115, 110, 175, 'and'), (0, 0, 105, 59, 'xor')]
Parsing finished in 0.045606374740600586 s
Generating circuit_3 ...
not (B and A) xor (not B)
Start parsing ...
[(0, 86, 105, 146, 'not'), (118, 44, 206, 103, 'xor'), (0, 0, 125, 60, 'notand')]
Parsing finished in 0.04700040817260742 s
Generating circuit_4 ...
(A and B) xor (B and A)
Start parsing ...
[(0, 0, 108, 60, 'and'), (0, 115, 108, 175, 'and'), (111, 58, 199, 117, 'xor')]
Parsing finished in 0.04537153244018555 s
Generating circuit_5 ...
(A and B) xor (not A)
Start parsing ...
[(0, 0, 115, 60, 'and'), (0, 86, 105, 146, 'not'), (118, 44, 206, 103, 'xor')]
Parsing finished in 0.050417184829711914 s
Generating circuit_6 ...
(A xor B) xor (B or A)
Star

In [10]:
import cv2
import numpy as np
from PIL import Image


def draw_bbox(img_path, output_path):
    # Load the image (grayscale from alpha channel)
    img = np.array(Image.open(img_path))[:, :, 3]

    # Load templates for all 7 gates
    template_and = np.array(Image.open('./and.png'))[:, :, 3]
    template_or = np.array(Image.open('./or.png'))[:, :, 3]
    template_not = np.array(Image.open('./not.png'))[:, :, 3]
    template_xor = np.array(Image.open('./xor.png'))[:, :, 3]
    template_notand = np.array(Image.open('./notand.png'))[:, :, 3]
    template_notor = np.array(Image.open('./notor.png'))[:, :, 3]
    template_notxor = np.array(Image.open('./notxor.png'))[:, :, 3]

    # Perform template matching for each gate
    result_and = cv2.matchTemplate(img, template_and, cv2.TM_CCOEFF_NORMED)
    result_or = cv2.matchTemplate(img, template_or, cv2.TM_CCOEFF_NORMED)
    result_not = cv2.matchTemplate(img, template_not, cv2.TM_CCOEFF_NORMED)
    result_xor = cv2.matchTemplate(img, template_xor, cv2.TM_CCOEFF_NORMED)
    result_notand = cv2.matchTemplate(img, template_notand, cv2.TM_CCOEFF_NORMED)
    result_notor = cv2.matchTemplate(img, template_notor, cv2.TM_CCOEFF_NORMED)
    result_notxor = cv2.matchTemplate(img, template_notxor, cv2.TM_CCOEFF_NORMED)

    # Define thresholds for each gate
    threshold_and = 0.94
    threshold_or = 0.94
    threshold_not = 0.94
    threshold_xor = 0.87
    threshold_notand = 0.87
    threshold_notor = 0.87
    threshold_notxor = 0.87

    # Find locations for matches above the thresholds
    loc_and = np.where(result_and >= threshold_and)
    loc_or = np.where(result_or >= threshold_or)
    loc_not = np.where(result_not >= threshold_not)
    loc_xor = np.where(result_xor >= threshold_xor)
    loc_notand = np.where(result_notand >= threshold_notand)
    loc_notor = np.where(result_notor >= threshold_notor)
    loc_notxor = np.where(result_notxor >= threshold_notxor)

    # Convert the image to 3-channel RGB to draw colored rectangles
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # Define colors for each gate
    color_and = (255, 0, 0)      # Blue for AND
    color_or = (0, 255, 0)       # Green for OR
    color_not = (0, 0, 255)      # Red for NOT
    color_xor = (255, 255, 0)    # Cyan for XOR
    color_notand = (255, 0, 255) # Magenta for NOTAND
    color_notor = (0, 255, 255)  # Yellow for NOTOR
    color_notxor = (128, 128, 128) # Gray for NOTXOR

    # Draw rectangles for each gate
    for pt in zip(*loc_and[::-1]):
        cv2.rectangle(img, pt, (pt[0] + template_and.shape[1], pt[1] + template_and.shape[0]), color_and, 2)
        print("识别到与门 (AND)")

    for pt in zip(*loc_or[::-1]):
        cv2.rectangle(img, pt, (pt[0] + template_or.shape[1], pt[1] + template_or.shape[0]), color_or, 2)
        print("识别到或门 (OR)")

    for pt in zip(*loc_not[::-1]):
        cv2.rectangle(img, pt, (pt[0] + template_not.shape[1], pt[1] + template_not.shape[0]), color_not, 2)
        print("识别到非门 (NOT)")

    for pt in zip(*loc_xor[::-1]):
        cv2.rectangle(img, pt, (pt[0] + template_xor.shape[1], pt[1] + template_xor.shape[0]), color_xor, 2)
        print("识别到异或门 (XOR)")

    for pt in zip(*loc_notand[::-1]):
        cv2.rectangle(img, pt, (pt[0] + template_notand.shape[1], pt[1] + template_notand.shape[0]), color_notand, 2)
        print("识别到与非门 (NOTAND)")

    for pt in zip(*loc_notor[::-1]):
        cv2.rectangle(img, pt, (pt[0] + template_notor.shape[1], pt[1] + template_notor.shape[0]), color_notor, 2)
        print("识别到或非门 (NOTOR)")

    for pt in zip(*loc_notxor[::-1]):
        cv2.rectangle(img, pt, (pt[0] + template_notxor.shape[1], pt[1] + template_notxor.shape[0]), color_notxor, 2)
        print("识别到异或非门 (NOTXOR)")

    # Save the resulting image with detected gates
    cv2.imwrite(output_path, img)

    # # Display the resulting image
    # cv2.imshow('Detected Circuit Components', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [13]:
for i in range(21, 25):
    draw_bbox(f"./circuits_output2/circuit_{i}.png", f"./circuits_output2_bbox/circuit_{i}.png")

识别到与门 (AND)
识别到或门 (OR)
识别到或门 (OR)
识别到异或门 (XOR)
识别到异或门 (XOR)
识别到与非门 (NOTAND)
识别到异或门 (XOR)
识别到异或门 (XOR)
识别到异或门 (XOR)
识别到与非门 (NOTAND)
识别到与门 (AND)
识别到与门 (AND)
识别到非门 (NOT)
识别到与非门 (NOTAND)
识别到与非门 (NOTAND)
识别到与门 (AND)
识别到或门 (OR)
识别到或门 (OR)
识别到异或门 (XOR)
识别到异或门 (XOR)
识别到与非门 (NOTAND)
